<a href="https://colab.research.google.com/github/jnickg/steganet/blob/main/steganogan_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import LeakyReLU

def steganogan_conv2d_layer(layer_in, num_filters, kernel_size, name=None, normalize=True, activation_fn=LeakyReLU()):
    model = Conv2D(num_filters, kernel_size, padding='same', activation=activation_fn, name=name)(layer_in)
    if normalize:
        normalize_name = f'{name}_normalize' if name is not None else None
        model = BatchNormalization(name=normalize_name)(model)
    return model

In [3]:
from keras.layers import Input
from keras.layers import Concatenate
from keras.layers import Add
from keras.activations import sigmoid

def steganogan_encoder_dense_model(W, H, D):
    """
    The BasicEncoder module takes an cover image and a data tensor and combines
    them into a steganographic image.
    Input: (N, 3, H, W), (N, D, H, W)
    Output: (N, 3, H, W)
    """
    input_image = Input(shape=(W, H, 3), name=f'image{W}x{H}x{3}')
    input_data  = Input(shape=(W, H, D), name=f'data{W}x{H}x{D}')

    image_preprocess = steganogan_conv2d_layer(input_image, 32, 3, name='image_preprocess')

    image_data_process_1_in = Concatenate(name='image_data_process_1_in')([image_preprocess, input_data])
    image_data_process_1 = steganogan_conv2d_layer(image_data_process_1_in, 32, 3, name='image_data_process_1')

    image_data_process_2_in = Concatenate(name='image_data_process_2_in')([image_preprocess, image_data_process_1, input_data])
    image_data_process_2 = steganogan_conv2d_layer(image_data_process_2_in, 32, 3, name='image_data_process_2')

    encoder_in = Concatenate(name='encoder_in')([image_preprocess, image_data_process_1, image_data_process_2, input_data])
    encoder = steganogan_conv2d_layer(encoder_in, 3, 3, name='encoder', normalize=False, activation_fn=sigmoid)

    encoder_out = Add(name='encoder_out')([input_image, encoder])

    return ([input_image, input_data], encoder_out)

In [4]:
from keras.models import Model

my_W = 128
my_H = 128
my_D = 2
steganofan_encoder_ins, steganofan_encoder_outs = steganogan_encoder_dense_model(my_W, my_H, my_D)

steganogan_encoder = Model(steganofan_encoder_ins, steganofan_encoder_outs, name='steganogan_encoder')
steganogan_encoder.summary()

Model: "steganogan_encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image128x128x3 (InputLayer)     [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
image_preprocess (Conv2D)       (None, 128, 128, 32) 896         image128x128x3[0][0]             
__________________________________________________________________________________________________
image_preprocess_normalize (Bat (None, 128, 128, 32) 128         image_preprocess[0][0]           
__________________________________________________________________________________________________
data128x128x2 (InputLayer)      [(None, 128, 128, 2) 0                                            
_________________________________________________________________________________

In [5]:
def steganogan_decoder_dense_model(W, H, D, input=None):
    """
    The DenseDecoder module takes an steganographic image and attempts to decode
    the embedded data tensor.
    Input: (N, 3, H, W)
    Output: (N, D, H, W)
    """
    if input is None:
        input = Input(shape=(W, H, 3), name=f'cover_image{W}x{H}x{3}')
    decode_1 = steganogan_conv2d_layer(input, 32, 3, name='decode_1')

    decode_2 = steganogan_conv2d_layer(decode_1, 32, 3, name='decode_2')

    decode_3_in = Concatenate(name='decode_3_in')([decode_1, decode_2])
    decode_3 = steganogan_conv2d_layer(decode_3_in, 32, 3, name='decode_3')

    decoder_in = Concatenate(name='decoder_in')([decode_1, decode_2, decode_3])
    decoder = steganogan_conv2d_layer(decoder_in, D, 3, name='decoder', normalize=False, activation_fn=sigmoid)

    return decoder

In [6]:
decoder_outs = steganogan_decoder_dense_model(my_W, my_H, my_D, input=steganofan_encoder_outs)

steganogan_full = Model(steganofan_encoder_ins, decoder_outs, name='steganogan_full')
steganogan_full.summary()

Model: "steganogan_full"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image128x128x3 (InputLayer)     [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
image_preprocess (Conv2D)       (None, 128, 128, 32) 896         image128x128x3[0][0]             
__________________________________________________________________________________________________
image_preprocess_normalize (Bat (None, 128, 128, 32) 128         image_preprocess[0][0]           
__________________________________________________________________________________________________
data128x128x2 (InputLayer)      [(None, 128, 128, 2) 0                                            
____________________________________________________________________________________

In [7]:
from keras.layers import AveragePooling2D

def steganogan_critic_model(W, H, D):
    """
    The BasicCritic module takes an image and predicts whether it is a cover
    image or a steganographic image (N, 1).
    Input: (N, 3, H, W)
    Output: (N, 1)
    """
    model_in = Input(shape=(W, H, 3), name=f'image{W}x{H}x{3}')
    model = steganogan_conv2d_layer(model_in, 32, 3, name='conv_1')
    model = steganogan_conv2d_layer(model, 32, 3, name='conv_2')
    model = steganogan_conv2d_layer(model, 32, 3, name='conv_3')
    model = steganogan_conv2d_layer(model, 32, 3, name='conv_4', normalize=False, activation_fn=sigmoid)
    model = AveragePooling2D(name='mean')(model)
    return model_in, model

In [8]:
#
# These probably need to be re-made as classes in order to properly
# behave as described in the paper. Subclass keras.Model
#

critic_in, critic_out = steganogan_critic_model(my_W, my_H, my_D)

steganogan_critic = Model(critic_in, critic_out, name='steganogan_critic')
steganogan_critic.summary()

Model: "steganogan_critic"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image128x128x3 (InputLayer)  [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv_1_normalize (BatchNorma (None, 128, 128, 32)      128       
_________________________________________________________________
conv_2 (Conv2D)              (None, 128, 128, 32)      9248      
_________________________________________________________________
conv_2_normalize (BatchNorma (None, 128, 128, 32)      128       
_________________________________________________________________
conv_3 (Conv2D)              (None, 128, 128, 32)      9248      
_________________________________________________________________
conv_3_normalize (BatchNorma (None, 128, 128, 32)

In [9]:
#